In [1]:
import gym
from room.envs import GymEnvWrapper
from typing import List, Optional, Union, Dict
from room.common.typing import CfgType
import torch
from tqdm import trange

from room import notice
from room.agents import Agent, DQN
from room.common.callbacks import Callback
from room.common.preprocessing import get_param
from room.envs.wrappers import EnvWrapper
from room.loggers import Logger
from room.memories import Memory
from room.trainers.base import Trainer
from room.agents.policies import Policy
from room.networks.blocks import MLP


/home/ksterx/Applications/miniconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env = gym.make("CartPole-v1", render_mode="human")
print(env.observation_space.shape)
env = GymEnvWrapper(env)

(4,)


In [17]:
class DQN(Agent):
    
    def __init__(
        self,
        policy: Union[Policy, str],
        cfg: CfgType,
        logger: Optional[Logger] = None,
        *args,
        **kwargs,
    ):
        super().__init__(policy, cfg, logger)
        self.policy = MLP([4, 100, 2], activation="relu")
        self.q_net = self.policy
        self.target_q_net = self.policy
        self.loss = None  # TODO: add loss

    def act(self, state: torch.Tensor) -> torch.Tensor:
        q = self.target_q_net(state)
        action = torch.argmax(q)

        # eposilon greedy
        if torch.rand(1) < 0.1:
            action = torch.randint(0, 2, (1,))
        return action

    def learn(self, batch: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        q = self.q_net(batch["state"])
        q = q.gather(1, batch["action"].unsqueeze(1)).squeeze(1)
        q_next = target_q_net(batch["next_state"])
        q_next = q_next.max(1)[0]
        q_target = batch["reward"] + cfg.agent.gamma * q_next
        loss = loss(q, q_target)


In [18]:
agent = DQN([], {})

[WARNING] Policy [] is not supported
Sequential(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=2, bias=True)
  (relu2): ReLU()
)


In [19]:
memory = Memory(capacity=10)

In [20]:
state = env.reset()

In [24]:
for t in trange(1000):
    with torch.no_grad():
        notice.debug(f"States: {state}")

        action = agent.act(state[0])
    next_states, rewards, terminated, truncated, info = env.step(actions)
    memory.add(
        {
            "states": states,
            "actions": actions,
            "rewards": rewards,
            "next_states": next_states,
            "terminated": terminated,
            "truncated": truncated,
            "info": info,
        }
    )

    with torch.no_grad():
        for agent in agents:
            agent.collect(new_states, action, reward)

    with torch.no_grad():
        if terminated.any() or truncated.any():
            states, infos = env.reset()

  0%|                                                                                                                                                                                                                                                                                                                                                                                                           | 0/1000 [00:00<?, ?it/s]

[DEBUG] States: (tensor([[-0.0375,  0.0410,  0.0461, -0.0188]], device='cuda:0'), {})


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)